In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras import layers, callbacks
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential,Model
from keras.layers import *
from keras import backend as K
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from sklearn.metrics import accuracy_score, cohen_kappa_score, roc_auc_score, matthews_corrcoef, average_precision_score
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import model_selection, metrics
from sklearn.model_selection import train_test_split,StratifiedKFold,KFold

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
rnnamp_model = pd.read_csv('./rnnamp.csv')

In [ ]:
m=0
clas=['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']
encode=[]
for i in range(len(rnnamp_model)):
    st=rnnamp_model['text'][i]
    row=[]
    ind=1
    c=0
    if 51>len(st):
      m=m+1
    for j in range(50):
        if j<len(st):
          if st[j] != ' ':
            index=clas.index(st[j])
            l=np.zeros(20)
            l[index]=1
          else:
            c=c+1
            continue
        else:
            l=np.zeros(20)
        row.append(l)
    l=np.zeros((c,20))
    row=np.array(row)
    row=np.append(row,l,axis=0)
    encode.append(row)
encode=np.array(encode)

In [ ]:
x_model1=encode
y_model1=np.array(rnnamp_model['labels'])
x_train1, x_test1, y_train1, y_test1= train_test_split(x_model1,y_model1, test_size=0.2, random_state=0)
x_train1, x_val1, y_train1, y_val1= train_test_split(x_train1,y_train1, test_size=0.25, random_state=0)

In [ ]:
skfold = StratifiedKFold(n_splits=10, random_state=0, shuffle= True)


In [ ]:
tf.random.set_seed(0)
np.random.seed(0)


In [ ]:
ls ='/content/drive/My Drive/best_model/'

In [ ]:

scores =[]
i=1
callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),tf.keras.callbacks.ModelCheckpoint(filepath=ls+'best_model1.h5', monitor='val_accuracy', save_best_only=True,mode='auto')]

model1 = Sequential()
model1.add(Conv2D(32, kernel_size=(11, 11),activation='relu',padding='same',input_shape=(50,20,1)))
model1.add(Conv2D(32, kernel_size=(11, 11),activation='relu',padding='same'))
model1.add(Conv2D(64, kernel_size=(7,7),activation='relu',padding='same'))
model1.add(Conv2D(64, kernel_size=(5,5),activation='relu',padding='same'))
model1.add(Conv2D(128, kernel_size=(3,3),activation='relu',padding='same'))
model1.add(Conv2D(128, kernel_size=(3,3),activation='relu',padding='same'))
model1.add(Flatten())
model1.add(tf.keras.layers.Dense(64, activation=tf.nn.relu,kernel_regularizer=tf.keras.regularizers.l2(l2=0.001)))
model1.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid,kernel_regularizer=tf.keras.regularizers.l2(l2=0.01)))
optimizer = Adam(learning_rate=0.0001)
model1.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
history = model1.fit(x_train1,y_train1,epochs = 100,batch_size=32,validation_data=(x_val1,y_val1),callbacks=[callback],verbose=1,shuffle= True)
y_pred=model1.predict(x_test1)
y_pred[y_pred>0.5]=1
y_pred[y_pred<0.5]=0
print('fold= ',i)
acc_model=100*metrics.accuracy_score(y_test1, y_pred)
print('Accuracy_model = %.3f' ,acc_model)
scores.append([acc_model])



Epoch 1/100
48/48 [==============================] - ETA: 0s - loss: 0.7692 - accuracy: 0.5114

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


48/48 [==============================] - 4s 35ms/step - loss: 0.7692 - accuracy: 0.5114 - val_loss: 0.7181 - val_accuracy: 0.5527
Epoch 2/100
48/48 [==============================] - 2s 31ms/step - loss: 0.7080 - accuracy: 0.5708 - val_loss: 0.6902 - val_accuracy: 0.6035
Epoch 3/100
48/48 [==============================] - 1s 31ms/step - loss: 0.6912 - accuracy: 0.6040 - val_loss: 0.6874 - val_accuracy: 0.6152
Epoch 4/100
48/48 [==============================] - 2s 38ms/step - loss: 0.6502 - accuracy: 0.6523 - val_loss: 0.6182 - val_accuracy: 0.6855
Epoch 5/100
48/48 [==============================] - 2s 33ms/step - loss: 0.6030 - accuracy: 0.7045 - val_loss: 0.5867 - val_accuracy: 0.7051
Epoch 6/100
48/48 [==============================] - 2s 32ms/step - loss: 0.5694 - accuracy: 0.7293 - val_loss: 0.5639 - val_accuracy: 0.7441
Epoch 7/100
48/48 [==============================] - 1s 30ms/step - loss: 0.5172 - accuracy: 0.7834 - val_loss: 0.5424 - val_accuracy: 0.7598
Epoch 8/100
48/48 

In [ ]:
model1 = tf.keras.models.load_model(ls+'rnnamp.h5')
y_pred=model1.predict(x_model1)
y_pred[y_pred>0.5]=1
y_pred[y_pred<0.5]=0

cv_preds = y_pred
print('HemoPI-1 model datasets')
name='deep learning'
print("%s: Accuracy %0.2f%%" % (name, 100*metrics.accuracy_score(y_model1, cv_preds)))
print("%s: Precision-Recall %0.2f%%" % (name, 100*metrics.average_precision_score(y_model1, cv_preds)))
print("%s: Matthews Coefficient %0.2f%%" % (name, 100*matthews_corrcoef(y_model1, cv_preds)))
print("%s: Cohen Kappa Score %0.2f%%" % (name, 100*metrics.cohen_kappa_score(y_model1, cv_preds)))
print("%s: ROC AUC Score %0.2f%%" % (name, 100*metrics.roc_auc_score(y_model1, cv_preds)))
target_names = ['low 0', 'high 1']
print(classification_report(y_model1, cv_preds, target_names=target_names))

# Predictions Validation Set
print('HemoPI-1 validation datasets')
y_pred2=model1.predict(x_test1)
y_pred2[y_pred2>0.5]=1
y_pred2[y_pred2<0.5]=0
cv_preds2 = y_pred2
print("%s: Accuracy %0.2f%%" % (name, 100*metrics.accuracy_score(y_test1, cv_preds2)))
print("%s: Precision %0.2f%%" % (name, 100*metrics.precision_score(y_test1, cv_preds2)))
print("%s: Recall %0.2f%%" % (name, 100*metrics.recall_score(y_test1, cv_preds2)))
print("%s: Matthews Coefficient %0.2f%%" % (name, 100*matthews_corrcoef(y_test1, cv_preds2)))
print("%s: Cohen Kappa Score %0.2f%%" % (name, 100*metrics.cohen_kappa_score(y_test1, cv_preds2)))
print("%s: ROC AUC Score %0.2f%%" % (name, 100*metrics.roc_auc_score(y_test1, cv_preds2)))
target_names = ['low 0', 'high 1']
print(classification_report(y_test1, cv_preds2, target_names=target_names))

80/80 [==============================] - 0s 5ms/step
HemoPI-1 model datasets
deep learning: Accuracy 91.40%
deep learning: Precision-Recall 89.17%
deep learning: Matthews Coefficient 82.77%
deep learning: Cohen Kappa Score 82.75%
deep learning: ROC AUC Score 91.43%
              precision    recall  f1-score   support

       low 0       0.90      0.92      0.91      1198
      high 1       0.93      0.91      0.92      1359

    accuracy                           0.91      2557
   macro avg       0.91      0.91      0.91      2557
weighted avg       0.91      0.91      0.91      2557

HemoPI-1 validation datasets
16/16 [==============================] - 0s 5ms/step
deep learning: Accuracy 80.66%
deep learning: Precision 83.00%
deep learning: Recall 78.95%
deep learning: Matthews Coefficient 61.42%
deep learning: Cohen Kappa Score 61.35%
deep learning: ROC AUC Score 80.73%
              precision    recall  f1-score   support

       low 0       0.78      0.83      0.80       246
     

In [ ]:
hlppredfuse_model = pd.read_csv('./hlppredfuse.csv')

In [ ]:
m=0
clas=['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']
encode2=[]
for i in range(len(hlppredfuse_model)):
    st=hlppredfuse_model['text'][i]
    row=[]
    ind=1
    c=0
    for j in range(50):
        if j<len(st):
          if st[j] != ' ':
            index=clas.index(st[j])
            l=np.zeros(20)
            l[index]=1
          else:
            c=c+1
            continue
        else:
            l=np.zeros(20)
        row.append(l)
    l=np.zeros((c,20))
    row=np.array(row)
    row=np.append(row,l,axis=0)
    encode2.append(row)
encode2=np.array(encode2)

In [ ]:
x_model2=encode2
y_model2=np.array(hlppredfuse_model['labels'])
x_train2, x_test2, y_train2, y_test2= train_test_split(x_model2,y_model2, test_size=0.2, random_state=0)
x_train2, x_val2, y_train2, y_val2 = train_test_split(x_train2,y_train2, test_size=0.25, random_state=0)


In [ ]:
scores =[]
i=1
callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),tf.keras.callbacks.ModelCheckpoint(filepath=ls+'best_model2.h5', monitor='val_accuracy',save_best_only=True,mode='auto')]

model2 = Sequential()
model2.add(Conv2D(32, kernel_size=(11, 11),activation='relu',padding='same',input_shape=(50,20,1)))
model2.add(Conv2D(32, kernel_size=(11, 11),activation='relu',padding='same'))
model2.add(Conv2D(64, kernel_size=(7,7),activation='relu',padding='same'))
model2.add(Conv2D(64, kernel_size=(5,5),activation='relu',padding='same'))
model2.add(Conv2D(128, kernel_size=(3,3),activation='relu',padding='same'))
model2.add(Conv2D(128, kernel_size=(3,3),activation='relu',padding='same'))
model2.add(Flatten())
model2.add(tf.keras.layers.Dense(64, activation=tf.nn.relu,kernel_regularizer=tf.keras.regularizers.l2(l2=0.001)))
model2.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid,kernel_regularizer=tf.keras.regularizers.l2(l2=0.01)))
optimizer = Adam(learning_rate=0.0001)
model2.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
history = model2.fit(x_train2,y_train2,epochs = 100,batch_size=32,validation_data=(x_val2,y_val2),callbacks=[callback],verbose=0,shuffle= True)
y_pred=model2.predict(x_test2)
y_pred[y_pred>0.5]=1
y_pred[y_pred<0.5]=0
print('fold= ',i)
acc_model=100*metrics.accuracy_score(y_test2, y_pred)
print('Accuracy_model %0.3f' ,100*metrics.accuracy_score(y_test2, y_pred))


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


22/22 [==============================] - 1s 14ms/step
fold=  1
Accuracy_model %0.3f 94.17613636363636


In [ ]:
model2 = tf.keras.models.load_model(ls+'hlppredfuse.h5')
y_pred=model2.predict(x_model2)
y_pred[y_pred>0.5]=1
y_pred[y_pred<0.5]=0

cv_preds = y_pred
print('HemoPI-2 model datasets')
name='deep learning'
print("%s: Accuracy %0.2f%%" % (name, 100*metrics.accuracy_score(y_model2, cv_preds)))
print("%s: Precision-Recall %0.2f%%" % (name, 100*metrics.average_precision_score(y_model2, cv_preds)))
print("%s: Matthews Coefficient %0.2f%%" % (name, 100*matthews_corrcoef(y_model2, cv_preds)))
print("%s: Cohen Kappa Score %0.2f%%" % (name, 100*metrics.cohen_kappa_score(y_model2, cv_preds)))
print("%s: ROC AUC Score %0.2f%%" % (name, 100*metrics.roc_auc_score(y_model2, cv_preds)))
tn, fp, fn, tp = confusion_matrix(y_model2, cv_preds).ravel()

# calculate sensitivity
sensitivity = tp / (tp + fn)
print("Sensitivity:", sensitivity)
specificity = tn / (tn + fp)
print("Specificity:", specificity)

target_names = ['low 0', 'high 1']
print(classification_report(y_model2, cv_preds, target_names=target_names))

# Predictions Validation Set
print('HemoPI-2 validation datasets')
y_pred2=model2.predict(x_test2)
y_pred2[y_pred2>0.5]=1
y_pred2[y_pred2<0.5]=0
cv_preds2 = y_pred2
print("%s: Accuracy %0.2f%%" % (name, 100*metrics.accuracy_score(y_test2, cv_preds2)))
print("%s: Precision-Recall %0.2f%%" % (name, 100*metrics.precision_score(y_test2, cv_preds2)))
print("%s: Matthews Coefficient %0.2f%%" % (name, 100*matthews_corrcoef(y_test2, cv_preds2)))
print("%s: Cohen Kappa Score %0.2f%%" % (name, 100*metrics.cohen_kappa_score(y_test2, cv_preds2)))
print("%s: ROC AUC Score %0.2f%%" % (name, 100*metrics.roc_auc_score(y_test2, cv_preds2)))
print("%s: Recall %0.2f%%" % (name, 100*metrics.recall_score(y_test2, cv_preds2)))

tn, fp, fn, tp = confusion_matrix(y_test2, cv_preds2).ravel()

# calculate sensitivity
sensitivity = tp / (tp + fn)
print("Sensitivity:", sensitivity)
specificity = tn / (tn + fp)
print("Specificity:", specificity)

target_names = ['low 0', 'high 1']
print(classification_report(y_test2, cv_preds2, target_names=target_names))

110/110 [==============================] - 77s 691ms/step
HemoPI-2 model datasets
deep learning: Accuracy 97.19%
deep learning: Precision-Recall 92.98%
deep learning: Matthews Coefficient 93.41%
deep learning: Cohen Kappa Score 93.38%
deep learning: ROC AUC Score 96.26%
Sensitivity: 0.9379562043795621
Specificity: 0.9872006606110653
              precision    recall  f1-score   support

       low 0       0.97      0.99      0.98      2422
      high 1       0.97      0.94      0.95      1096

    accuracy                           0.97      3518
   macro avg       0.97      0.96      0.97      3518
weighted avg       0.97      0.97      0.97      3518

HemoPI-2 validation datasets
22/22 [==============================] - 15s 683ms/step
deep learning: Accuracy 94.89%
deep learning: Precision-Recall 94.66%
deep learning: Matthews Coefficient 87.99%
deep learning: Cohen Kappa Score 87.89%
deep learning: ROC AUC Score 93.18%
deep learning: Recall 88.64%
Sensitivity: 0.8863636363636364
Spe

In [ ]:
combined_model = pd.read_csv('./combined.csv')

In [ ]:
m=0
clas=['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']
encode3=[]
for i in range(len(combined_model)):
    st=combined_model['text'][i]
    row=[]
    ind=1
    c=0
    for j in range(50):
        if j<len(st):
          if st[j] != ' ':
            index=clas.index(st[j])
            l=np.zeros(20)
            l[index]=1
          else:
            c=c+1
            continue
        else:
            l=np.zeros(20)
        row.append(l)
    l=np.zeros((c,20))
    row=np.array(row)
    row=np.append(row,l,axis=0)
    encode3.append(row)
encode3=np.array(encode3)

In [ ]:
x_model3=encode3
y_model3=np.array(combined_model['labels'])
x_train3, x_test3, y_train3, y_test3 = train_test_split(x_model3,y_model3, test_size=0.2, random_state=0)
x_train3, x_val3, y_train3, y_val3 = train_test_split(x_train3,y_train3, test_size=0.25, random_state=0)


In [ ]:
i=1
callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),tf.keras.callbacks.ModelCheckpoint(filepath=ls+'best_model3.h5', monitor='val_accuracy', save_best_only=True,mode='auto')]

model3 = Sequential()
model3.add(Conv2D(32, kernel_size=(11, 11),activation='relu',padding='same',input_shape=(50,20,1)))
model3.add(Conv2D(32, kernel_size=(11, 11),activation='relu',padding='same'))
model3.add(Conv2D(64, kernel_size=(7,7),activation='relu',padding='same'))
model3.add(Conv2D(64, kernel_size=(5,5),activation='relu',padding='same'))
model3.add(Conv2D(128, kernel_size=(3,3),activation='relu',padding='same'))
model3.add(Conv2D(128, kernel_size=(3,3),activation='relu',padding='same'))
model3.add(Flatten())
model3.add(tf.keras.layers.Dense(64, activation=tf.nn.relu,kernel_regularizer=tf.keras.regularizers.l2(l2=0.001)))
model3.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid,kernel_regularizer=tf.keras.regularizers.l2(l2=0.01)))
optimizer = Adam(learning_rate=0.0001)
model3.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
history = model3.fit(x_train3,y_train3,epochs = 100,batch_size=32,validation_data=(x_val3,y_val3),callbacks=[callback],verbose=0,shuffle=True)
y_pred=model3.predict(x_test3)
y_pred[y_pred>0.5]=1
y_pred[y_pred<0.5]=0
print('fold= ',i)
acc_model=100*metrics.accuracy_score(y_test3, y_pred)
print('Accuracy_model %0.3f' ,100*metrics.accuracy_score(y_test3, y_pred))


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


45/45 [==============================] - 0s 5ms/step
fold=  1
Accuracy_model %0.3f 88.09192200557104


In [ ]:
model3 = tf.keras.models.load_model(ls+'combined.h5')
y_pred=model3.predict(x_model3)
y_pred[y_pred>0.5]=1
y_pred[y_pred<0.5]=0

cv_preds = y_pred
print('HemoPI-3 model datasets')
name='deep learning'
print("%s: Accuracy %0.2f%%" % (name, 100*metrics.accuracy_score(y_model3, cv_preds)))
print("%s: Precision-Recall %0.2f%%" % (name, 100*metrics.average_precision_score(y_model3, cv_preds)))
print("%s: Matthews Coefficient %0.2f%%" % (name, 100*matthews_corrcoef(y_model3, cv_preds)))
print("%s: Cohen Kappa Score %0.2f%%" % (name, 100*metrics.cohen_kappa_score(y_model3, cv_preds)))
print("%s: ROC AUC Score %0.2f%%" % (name, 100*metrics.roc_auc_score(y_model3, cv_preds)))
tn, fp, fn, tp = confusion_matrix(y_model3, cv_preds).ravel()

# calculate sensitivity
sensitivity = tp / (tp + fn)
print("Sensitivity:", sensitivity)
specificity = tn / (tn + fp)
print("Specificity:", specificity)

target_names = ['low 0', 'high 1']
print(classification_report(y_model3, cv_preds, target_names=target_names))

# Predictions Validation Set
print('HemoPI-3 validation datasets')
y_pred2=model3.predict(x_test3)
y_pred2[y_pred2>0.5]=1
y_pred2[y_pred2<0.5]=0
cv_preds2 = y_pred2
print("%s: Accuracy %0.2f%%" % (name, 100*metrics.accuracy_score(y_test3, cv_preds2)))
print("%s: Precision-Recall %0.2f%%" % (name, 100*metrics.average_precision_score(y_test3, cv_preds2)))
print("%s: Matthews Coefficient %0.2f%%" % (name, 100*matthews_corrcoef(y_test3, cv_preds2)))
print("%s: Cohen Kappa Score %0.2f%%" % (name, 100*metrics.cohen_kappa_score(y_test3, cv_preds2)))
print("%s: ROC AUC Score %0.2f%%" % (name, 100*metrics.roc_auc_score(y_test3, cv_preds2)))
tn, fp, fn, tp = confusion_matrix(y_test3, cv_preds2).ravel()

# calculate sensitivity
sensitivity = tp / (tp + fn)
print("Sensitivity:", sensitivity)
specificity = tn / (tn + fp)
print("Specificity:", specificity)

target_names = ['low 0', 'high 1']
print(classification_report(y_test3, cv_preds2, target_names=target_names))

225/225 [==============================] - 1s 5ms/step
HemoPI-3 model datasets
deep learning: Accuracy 94.97%
deep learning: Precision-Recall 90.96%
deep learning: Matthews Coefficient 89.66%
deep learning: Cohen Kappa Score 89.66%
deep learning: ROC AUC Score 94.80%
Sensitivity: 0.9371466577984703
Specificity: 0.9587727708533078
              precision    recall  f1-score   support

       low 0       0.95      0.96      0.96      4172
      high 1       0.94      0.94      0.94      3007

    accuracy                           0.95      7179
   macro avg       0.95      0.95      0.95      7179
weighted avg       0.95      0.95      0.95      7179

HemoPI-3 validation datasets
45/45 [==============================] - 0s 5ms/step
deep learning: Accuracy 87.81%
deep learning: Precision-Recall 79.04%
deep learning: Matthews Coefficient 74.84%
deep learning: Cohen Kappa Score 74.83%
deep learning: ROC AUC Score 87.32%
Sensitivity: 0.8439597315436241
Specificity: 0.9023809523809524
      